## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Импортируем необходимые библиотеки

In [15]:
import pandas as pd
from collections import Counter
from pymystem3 import Mystem

In [16]:
#Изучим информацию
m=Mystem()
#df = pd.read_csv('/Users/vladimrogulo/PycharmProjects/Praktikum1/data.csv')
df = pd.read_csv('/datasets/data.csv')
df.info()
display(df.head(10))
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод

#### Читаем CSV файл по указанному пути и просмативаем сводную информацию
**children** — количество детей в семье (*Имеются отрицательные значения*)

**days_employed** — общий трудовой стаж в днях (*Имеюстя пропущенные значения. Имеются отрицательные значения. Неверный тип данных*)

**dob_years** — возраст клиента в годах

**education** — уровень образования клиента 

**education_id** — идентификатор уровня образования

**family_status** — семейное положение

**family_status_id** — идентификатор семейного положения

**gender** — пол клиента

**income_type** — тип занятости

**debt** — имел ли задолженность по возврату кредитов

**total_income** — ежемесячный доход (*Имеюстя пропущенные значения*)

**purpose** — цель получения кредита

Требуется привести строки к нижнему регистру, заполнить пропущенные значения, отрицательные 
значения привести к положительным (делаем допущения, что "-" появился при выгрузке данных).
Учитывая, что в двух столбцах одинаковое количество пропусков, можно сделать вывод, что пропуска неслучайны. Минимальное значение возраста клиета равно 0, что является артефактом.(Наше исследование не затрагивает зависимость к возраст, соответственно обрабатывать мы его не будем)


### Шаг 2. Предобработка данных

### Обработка пропусков

In [17]:
df['days_employed'] = abs(df['days_employed']) # заменяем отрицательные значения в трудовом стаже 
df['children'] = abs(df['children'])# заменяем отрицательные значения в "количестве детей"
print(df['children'].value_counts())# Максимальное количество детей - 20 (76 позиций). Предположим, что форма не удобная и пользователь хотел указать 2 детей
df['children'] = df['children'].replace(20, 2)

# Приводим значения к нижнему регистру
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df['gender'] = df['gender'].str.lower()
df['income_type'] = df['income_type'].str.lower()
display(df.head(10))




0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,m,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,f,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425.938277,покупка жилья для семьи


#### Определим зависимость пропусков в столбцах  days_employed и total_income

In [18]:
# Выводим количество пустых строк в столбце days_employed
print('Количество пропущенных строк в стобце days_employed:', df['days_employed'].isnull().sum()) 
# Выводим количество пустых строк в столбце total_income
print('Количество пропущенных строк в стобце total_income:', df['total_income'].isnull().sum())

# Выберем в df строки, где в столбце days_employed пропущены значения
# и посчитаем количество нулевых значений в столбце total_income
print('Количество совпадений пустых строк:',df[(df['days_employed'].isnull() == True)]['total_income'].isnull().sum()) 

# Выведем на экран строки 10 строк с пропущенным значениями для анализа
display(df[(df['days_employed'].isnull() == True) & (df['total_income'].isnull() == True)].head(10))
# Пропуски носят случайный характер



Количество пропущенных строк в стобце days_employed: 2174
Количество пропущенных строк в стобце total_income: 2174
Количество совпадений пустых строк: 2174


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,m,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,m,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,не женат / не замужем,4,f,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,f,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,f,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,не женат / не замужем,4,m,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,f,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,m,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,f,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,m,сотрудник,0,NaN,жилье


#### Заполним пропущенные значения

In [19]:
# В пустые ячейки стажа работы и ежемесячного дохода вставим значения медиан по типу занятости
# Рассмотрим имеющиеся значения в таблице income_type
display(df['income_type'].value_counts())
job_types = df['income_type'].unique()# Мы имеем 8 уникальнмх типов занятости
# Функция для определения медианы и заполнения столбца полученным значением
def fill_income_median(data, job_type):
    income_median = df[df['income_type'] == job_type]['total_income'].median()
    data.loc[data['income_type'] == job_type,'total_income'] = data.loc[data['income_type'] == job_type,'total_income'].fillna(income_median)

# Использование функции для 8 уникальных типов занятости
for job_type in job_types:
    fill_income_median(df, job_type)

df.info()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

После опредления зависимости проспусков в столбцах days_employed и total_income можно сделать вывод, 
что пропуски не случайны. Отсутвие указанных данных о ежемесячном доходе напрямую зависит от пропуска в графе "трудовой 
стаж".
Для 8 уникальных видов занятости заполнили пропущенные значения медианой по срезу.

### Замена типа данных

In [20]:
# Заполним пропуски 0
df['days_employed'].fillna(0, inplace=True)
# Приведем стаж работы к целому числу 
df['days_employed']=df['days_employed'].astype('int')
df.info() # Проверяем


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод

Произвели замену пропущенных значений на нулевые. Перевели к типу 'int', так как Стаж измеряется в днях


### Обработка дубликатов

In [21]:
# Определим есть ли дубликаты и их количестово
print('Количество дубликатов:', df.duplicated().sum()) 
# Убираем дубликаты методом drop_duplicates
df.drop_duplicates(inplace=True)
print(df.info()) # проверим результат

Количество дубликатов: 71
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  int64  
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB
None


### Вывод

Очистили датасет от дубликатов. Дубликаты могли появится при сбоях при выгрузки данных, либо при повторной отправки формы при заполнении анкеты пользователем.

### Лемматизация

In [22]:
# Начнем с функции для лемматизации столбца
def purpose_lem(purpose):
    lemmas = m.lemmatize(purpose)
    
    return lemmas
# Добавление ногового столбца с помощью применения вышеуказанной функции purpose_lem
df['lemma_purpose']=df['purpose'].apply(purpose_lem)
display(df.head(10))
# Найдем все уникальные цели и количество их повторений для лемматизированного списка
purposes = ' '.join(df['purpose'].unique())
result = m.lemmatize(purposes)
print(Counter(result))
# Сформируем список высокочастотных категрий из выдачи выше
purpose_result = ['недвижимость', 'автомобиль', 'образование', 'жилье', 'свадьба']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemma_purpose
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу,"[сыграть, , свадьба, \n]"
5,0,926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763.565419,покупка жилья,"[покупка, , жилье, \n]"
6,0,2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525.971920,операции с жильем,"[операция, , с, , жилье, \n]"
7,0,152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823.934197,образование,"[образование, \n]"
8,2,6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856.832424,на проведение свадьбы,"[на, , проведение, , свадьба, \n]"
9,0,2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425.938277,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]"


Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


### Вывод

Проведена лемматизации столбца "purpose" (цель кредита), результа был выделен в новый столбец.
Выведен на экран подсчет уникальных значений в списке лемматизированных слов. Рассмотрев высокочастотные леммы, выделили основные цели кредитов. 


### Категоризация данных

In [23]:
# Напришем функцию для категоризации целей покупки
def purpose_category(lemma_purpose):
    for purpose in purpose_result:
        if purpose in lemma_purpose:
            return purpose
        
# Напишем Функцию для категоризации дохода. Значения взяты 25% и 75% столбца total_income с таблицы при вывове функции describe
def cat_income(total_income):
    if total_income < 100000:
        return 'низкий'
    if total_income < 200000:
        return 'средний'
    return 'высокий'
    
# Добавление ногового столбца с помощью применения вышеуказанной функции purpose_category
df['purpose_category']=df['lemma_purpose'].apply(purpose_category)
df['cat_income']=df['total_income'].apply(cat_income)

display(df.head(10))



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemma_purpose,purpose_category,cat_income
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья,"[покупка, , жилье, \n]",жилье,высокий
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,средний
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья,"[покупка, , жилье, \n]",жилье,средний
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование,"[дополнительный, , образование, \n]",образование,высокий
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,средний
5,0,926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763.565419,покупка жилья,"[покупка, , жилье, \n]",жилье,высокий
6,0,2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525.971920,операции с жильем,"[операция, , с, , жилье, \n]",жилье,высокий
7,0,152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823.934197,образование,"[образование, \n]",образование,средний
8,2,6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856.832424,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба,низкий
9,0,2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425.938277,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",жилье,средний


### Вывод

Разбили клиентов по целям кредитов.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [24]:
children_debt_mean = (df.groupby('children')[['debt']].agg(['mean', 'sum','count']))

display(children_debt_mean)

debt             
              mean   sum  count
children                       
0         0.075438  1063  14091
1         0.091658   445   4855
2         0.094925   202   2128
3         0.081818    27    330
4         0.097561     4     41
5         0.000000     0      9

### Вывод

Существует зависимость между наличием детей и возвратом кредита в срок. Так, среди людей без детей 7,5% людей не возвращают кредит.

Среди заемщиков с 1-2 детьми процент невозврата кредита выше - 9,1% и 9,4% соответвенно. 

Для того, чтобы сделать аналогичный вывод  по заемщикам с 3 и более детьми имеющихся данных недостаточно.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [25]:
family_debt_mean = (df.groupby('family_status')[['debt']].agg(['mean', 'sum','count']))
display(family_debt_mean)

debt            
                           mean  sum  count
family_status                              
в разводе              0.071130   85   1195
вдовец / вдова         0.065693   63    959
гражданский брак       0.093471  388   4151
женат / замужем        0.075452  931  12339
не женат / не замужем  0.097509  274   2810

### Вывод

Люди, состоящиеся в гражданском браке(9,3%) и не женатые/не замужем(9,7%) чаще пропускают дату выплаты кредита нежели люди в разводе(7,1%), 
вдовцы/вдовы(6,5%), женатые/замужем(7,5%)


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [26]:
dob_years = pd.cut(df['dob_years'], [0, 18, 40, 75])
total_income = pd.qcut(df['total_income'],4)
total_income_debt_mean=df.pivot_table('debt', index='income_type', columns = [total_income])
display(total_income_debt_mean)
total_cat_debt_mean=df.groupby('cat_income')[['debt']].agg(['mean', 'sum','count'])
display(total_cat_debt_mean)

total_income,"(20667.263, 107623.857]","(107623.857, 142594.397]","(142594.397, 195820.926]","(195820.926, 2265604.029]"
income_type,,,,
безработный,1.000000,NaN,NaN,0.000000
в декрете,1.000000,NaN,NaN,NaN
госслужащий,0.068376,0.081395,0.048035,0.048718
компаньон,0.082777,0.079327,0.079011,0.063622
пенсионер,0.047586,0.065004,0.057489,0.060504
предприниматель,NaN,NaN,NaN,0.000000
сотрудник,0.096051,0.098898,0.104049,0.083169
студент,0.000000,NaN,NaN,NaN


debt             
                mean   sum  count
cat_income                       
высокий     0.070653   358   5067
низкий      0.079319   354   4463
средний     0.086297  1029  11924

### Вывод

Заемщики с большей зарплатой чаще возвращают кредит в срок. 
Исключение состаляют пенсионеры, где процент варьируется от 4,7% до 6%.

Люди со средним уровнем дохода чаще пропускают дату выплат. Это может быть обусловлено тем, что заемщики со средним доходом, могут получить кредит на большую сумму, чем с низким доходом. Соответвенно размер ежемесячных выплат выше. При каких-либо форс-мажорных обстоятельствах крупную сумму тяжелее гасить в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [27]:
# Сводная таблица по образованию 
education_debt_mean=df.pivot_table('debt', index= 'education', columns = 'purpose_category')
display(education_debt_mean)

# Сгруппируем по целям на кредит
purpose_debt_mean = (df.groupby('purpose_category')[['debt']].agg(['mean', 'sum','count']))
display(purpose_debt_mean)

purpose_category,автомобиль,жилье,недвижимость,образование,свадьба
education,,,,,
высшее,0.064672,0.041479,0.052833,0.058887,0.045000
начальное,0.109091,0.120000,0.106383,0.090909,0.142857
неоконченное высшее,0.104575,0.103226,0.080000,0.091603,0.075000
среднее,0.102547,0.076312,0.081445,0.103043,0.092203
ученая степень,NaN,0.000000,0.000000,0.000000,NaN


debt           
                      mean  sum count
purpose_category                     
автомобиль        0.093590  403  4306
жилье             0.069058  308  4460
недвижимость      0.074634  474  6351
образование       0.092200  370  4013
свадьба           0.080034  186  2324

### Вывод

Минимальный процент по задержке выплат по кредиту имеют люди с высшим образованием. Наивысший  - люди с начальным или неоконченным высшим образованием.
9,3% не возвращают кредит в срок за автомобиль, что является наивысшим показателем.
Также 9,2% процента не возвращают с срок деньги, которые они брали на образование. Люди с начальным, среднем и неоконченнм высшим образованием имеют долги по кредиту  в 9%-10% случаев

### Шаг 4. Общий вывод

Получив датасет, мы определили и заполнили пропущенные значения. Все значения привели к нижнему регистру и заменили типы данных в сстолбцах, где требовалось. 
Далее мы нашли и удалили дубликаты и выделили леммы в значениях столбца с целями получения кредита.

Полученные данные мы категоризировали по доходу и целям целям на кредит.

Наивысшие показатели по невозврату кредита в срок имеют люди без высшего образования. Что касается пенсионеров, то : "Чем больше средства, тем трудней жить по средствам."
Категории, которые имеют больший процент по задолжности: автомобиль и образование.
Так как для того, чтобы взять кредит на жилье, требуется первоначальный взнос, люди более осознанно берут кредит.
Люди с большим доходом или же без детей чаще врзвращают кредит в срок.
Люди, состоящиеся в гражданском браке и не женатые(не замужем) чаще пропускают дату выплаты кредита.

Закончим мы цитатой Уильяма Федера: "Бюджет говорит вам, чего вы не можете себе позволить, но помешать вам купить это он не может."



### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.